In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!python -m spacy download el_core_news_sm

In [3]:
nlp = spacy.load('el_core_news_sm')

In [4]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [5]:
start_page = 0
end_page = 18

start_url = 'https://www.efsyn.gr/search?keywords=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&field_category=All&field_author=All&field_anonymous_author=&created%5B0%5D=&created%5B1%5D=&sort_by=created&page='

test_url = start_url + str(start_page)

test_url

'https://www.efsyn.gr/search?keywords=%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82&field_category=All&field_author=All&field_anonymous_author=&created%5B0%5D=&created%5B1%5D=&sort_by=created&page=0'

In [6]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [7]:
teaser_articles_list = doc.find('div', {'class':'default-teaser triple'}).find_all('article')

In [ ]:
teaser_articles_list

In [9]:
link = teaser_articles_list[0].find('a', {'class': 'full-link'})['href']

link

'/stiles/apopseis/396023_ah-ellada-sagapo'

In [10]:
main_url = 'https://www.efsyn.gr'
article_link = main_url + link
print(article_link)

https://www.efsyn.gr/stiles/apopseis/396023_ah-ellada-sagapo


In [11]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('div', {'class':'default-teaser triple'}).find_all('article')


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a', {'class': 'full-link'})['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [12]:
efsyn_teasers_df = pd.DataFrame(teasers_list)
efsyn_teasers_df

,url
0,https://www.efsyn.gr/stiles/apopseis/396023_ah...
1,https://www.efsyn.gr/themata/thema-tis-efsyn/3...
2,https://www.efsyn.gr/tehnes/media/386894_ta-me...
3,https://www.efsyn.gr/stiles/apopseis/384306_mi...
4,https://www.efsyn.gr/kosmos/eyropi/370145_akro...
...,...
223,https://www.efsyn.gr/themata/thema-tis-efsyn/2...
224,https://www.efsyn.gr/efsyn-city/synenteyxeis/2...
225,https://www.efsyn.gr/ellada/ygeia/220057_i-omo...
226,https://www.efsyn.gr/efsyn-city/synenteyxeis/2...


In [13]:
efsyn_teasers_df.to_csv('/content/temp/efsyn_teasers.csv', index=False)

In [14]:
article_url = 'https://www.efsyn.gr/stiles/apopseis/296867_titloi-teloys-kai-arhis'

In [15]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [16]:
article = doc.find('section', {'class': 'article__main'})

In [17]:
section = doc.find('a', {'class': 'article__category'}).text
section

'ΑΠΟΨΕΙΣ'

In [18]:
title = article.find('h1').text
title

'Τίτλοι τέλους και αρχής'

In [19]:
date = article.find('time')['datetime']
date

'2021-06-03 16:00'

In [20]:
author = article.find('div', {'class', "article__author"}).span.text
author

' Μαρία Μαθιουδάκη* '

In [21]:
paragraphs = article.find_all('p')
paragraphs

[<p title="Τίτλος">Πλησιάζει άραγε η αρχή του τέλους του διαχωρισμού των ανθρώπων; Κι αν ναι, αυτή είναι η αισιόδοξη ή η απαισιόδοξη οπτική; Κάθε εποχή και διαφορετικοί διαχωρισμοί: πλούσιοι - φτωχοί, βενιζελικοί - βασιλικοί, δεξιοί - αριστεροί, κόκκινοι - πράσινοι - μπλε και ασπρόμαυροι, εμπορικοί - ποιοτικοί. Τώρα, εμβολιασμένοι - ανεμβολίαστοι. Και οι διαμάχες καλά κρατούν.</p>,
 <p title="Κείμενο (2)">Κάποιοι, πλέον, για να μη μιλήσουν για διαχωρισμούς, κρύβουν τους λόγους τους πίσω από την έννοια των «επιλογών». Ολα είναι επιλογές, λένε. Κάποιοι σε μια αλλαγή της κατάστασης επιλέγουν να γυρίσουν τα φυλλαράκια τους προς τον ήλιο και να εξελιχθούν, ενώ άλλοι δεν μπορούν να αντέξουν στο ξεβόλεμα και μαραζώνουν. Ομως κι αυτή η θεωρία δεν εμπεριέχει τον μεγαλύτερο διαχωρισμό; Οι ανθεκτικοί και οι αποφασιστικοί πάντοτε θα έχουν το προτέρημα σε έναν κόσμο όπου το μέλλον μας καθορίζεται από την τοποθεσία όπου «ανθίζει» ο καθένας.</p>,
 <p title="Κείμενο (3)">Κι αν σε ξεριζώσουν; Τι κάνεις

In [22]:
p_texts_list = []                      # δημιουργία κενής λίστας για να αποθηκεύσουμε το text κάθε παραγράφου
paragraphs = article.find_all('p')     # λίστα με όλες τις παραγράφους
for p in paragraphs:                   # για κάθε μια από τις παραγράφους στη λίστα
  p_texts_list.append(p.text)          # πάρε το .text και πρόσθεσέ το στη λίστα με τα texts των παραγράφων
full_text = ' '.join(p_texts_list)     # συνένωση των texts των παραγράφων
full_text = "".join(full_text.splitlines())
full_text

'Πλησιάζει άραγε η αρχή του τέλους του διαχωρισμού των ανθρώπων; Κι αν ναι, αυτή είναι η αισιόδοξη ή η απαισιόδοξη οπτική; Κάθε εποχή και διαφορετικοί διαχωρισμοί: πλούσιοι - φτωχοί, βενιζελικοί - βασιλικοί, δεξιοί - αριστεροί, κόκκινοι - πράσινοι - μπλε και ασπρόμαυροι, εμπορικοί - ποιοτικοί. Τώρα, εμβολιασμένοι - ανεμβολίαστοι. Και οι διαμάχες καλά κρατούν. Κάποιοι, πλέον, για να μη μιλήσουν για διαχωρισμούς, κρύβουν τους λόγους τους πίσω από την έννοια των «επιλογών». Ολα είναι επιλογές, λένε. Κάποιοι σε μια αλλαγή της κατάστασης επιλέγουν να γυρίσουν τα φυλλαράκια τους προς τον ήλιο και να εξελιχθούν, ενώ άλλοι δεν μπορούν να αντέξουν στο ξεβόλεμα και μαραζώνουν. Ομως κι αυτή η θεωρία δεν εμπεριέχει τον μεγαλύτερο διαχωρισμό; Οι ανθεκτικοί και οι αποφασιστικοί πάντοτε θα έχουν το προτέρημα σε έναν κόσμο όπου το μέλλον μας καθορίζεται από την τοποθεσία όπου «ανθίζει» ο καθένας. Κι αν σε ξεριζώσουν; Τι κάνεις; Εδώ η θεωρία της επιλογής πέφτει σε τοίχο. Αν είσαι παιδάκι 5 χρόνων και α

In [23]:
full_articles_list = []

for article_url in efsyn_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('section', {'class': 'article__main'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "efsyn.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('a', {'class': 'article__category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = title = article.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  # date
  try:
    date = article.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('div', {'class', "article__author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [24]:
efsyn_full_articles_df = pd.DataFrame(full_articles_list)
efsyn_full_articles_df

,site,url,section,title,date,author,full_text
0,efsyn.gr,https://www.efsyn.gr/stiles/apopseis/396023_ah...,ΑΠΟΨΕΙΣ,Αχ Ελλάδα σ᾽αγαπώ,2023-07-01 16:00,Θανάσης Βασιλείου,Η πιο γλυκιά πατρίδα είναι η καρδιά…που τ’ άγι...
1,efsyn.gr,https://www.efsyn.gr/themata/thema-tis-efsyn/3...,ΤΟ ΘΕΜΑ ΤΗΣ «ΕΦ.ΣΥΝ.»,Η μαύρη βίβλος για τα δικαιώματα - Έργα και ημ...,2023-05-21 12:18,"Ντίνα Δασκαλοπούλου, Δημήτρης Τερζής, Γιώτα Τ...",Με το αφήγημα του «νόμου και της τάξης» και με...
2,efsyn.gr,https://www.efsyn.gr/tehnes/media/386894_ta-me...,MEDIA,Τα μέσα κοινωνικής δικτύωσης δεν είναι μια «ει...,2023-04-23 06:00,Δημήτρης Κανελλόπουλος,"«Θα ήταν πραγματικά πολύ περίεργο, στην εποχή ..."
3,efsyn.gr,https://www.efsyn.gr/stiles/apopseis/384306_mi...,ΑΠΟΨΕΙΣ,Μια περίεργη σύμπτωση απόψεων,2023-04-01 16:00,Γιώργος Γιαννουλόπουλος,"Υπάρχει μέτωπο αντι-ΣΥΡΙΖΑ; Και ναι, και όχι. ..."
4,efsyn.gr,https://www.efsyn.gr/kosmos/eyropi/370145_akro...,ΕΥΡΩΠΗ,Ακροδεξιοί τρομοκράτες ετοίμαζαν πραξικόπημα,2022-12-08 06:00,Βίκυ Καπετανοπούλου,"Ο ηγέτης της οργάνωσης, «πρίγκιπας» Χάινριχ Ρό..."
...,...,...,...,...,...,...,...
223,efsyn.gr,https://www.efsyn.gr/themata/thema-tis-efsyn/2...,ΤΟ ΘΕΜΑ ΤΗΣ «ΕΦ.ΣΥΝ.»,"Ευεξία, ομορφιά και «ευ ζην» στην πανδημία: η ...",2020-05-03 11:40,"Ντίνα Δασκαλοπούλου, Αφροδίτη Τζιαντζή",Καθώς ξετυλίγεται το νέο κεφάλαιο της ζωής μετ...
224,efsyn.gr,https://www.efsyn.gr/efsyn-city/synenteyxeis/2...,ΣΥΝΕΝΤΕΥΞΕΙΣ,Στ. Ανατολίτης: Το χιούμορ δεν έχει ηθική πυξί...,2019-12-09 16:33,Γιάννης Καψάσκης,Είναι ένας από τους σύγχρονους εκπροσώπους της...
225,efsyn.gr,https://www.efsyn.gr/ellada/ygeia/220057_i-omo...,ΥΓΕΙΑ,"«Η ομοιοπαθητική δεν είναι πολυτέλεια, απευθύν...",2019-11-22 22:30,Αφροδίτη Τζιαντζή,Σήμερα ξεκινάει τις εργασίες του στο Εθνικό Ιδ...
226,efsyn.gr,https://www.efsyn.gr/efsyn-city/synenteyxeis/2...,ΣΥΝΕΝΤΕΥΞΕΙΣ,«Μας δίνει δύναμη ότι παιδιά πιάνουν τις κιθάρ...,2019-11-18 10:39,Γιάννης Καψάσκης,Το να κυκλοφορήσει ένα rock συγκρότημα έναν αξ...


In [25]:
efsyn_full_articles_df.to_csv('/content/temp/efsyn_full_articles.csv', index=False)